#**Decision Tree - Classification (Scratch)**

**Threshold Criteria for Decision Tree**
1. Mean
  - Mean Value: Similar to the median, you can use the mean (average) of the feature values as the threshold. However, this method is more sensitive to outliers.
2. Percentiles
  - Specific Percentiles: You can select thresholds based on specific percentiles (e.g., 25th, 75th). This can be useful for certain applications where you want to create splits at specific intervals.
3. Custom Criteria
  - Domain Knowledge: Sometimes, domain knowledge can guide the selection of thresholds based on meaningful values specific to the dataset (e.g., physiological thresholds in medical data).
4. Statistical Tests
  - Statistical Tests: Using statistical methods like t-tests or ANOVA to determine significant differences between groups can help identify potential thresholds.
5. Quantile Thresholds
  - Equally Sized Groups: Divide the data into quantiles (e.g., quartiles or quintiles) to determine thresholds that create equally sized groups, which can provide a balanced split.
6. Entropy-Based or Gini-Based Thresholds
  - Recursive Evaluation: Instead of using a single median or mean, you can evaluate each possible value for a feature in a range of values and calculate the information gain (or Gini impurity reduction) for each threshold. The threshold yielding the highest information gain is selected.
7. Tree Ensemble Methods
  - Bootstrap Aggregation: In methods like Random Forest, multiple trees are built, and thresholds are averaged across trees to create a more robust model.
8. Dynamic Thresholds
  - Adapt to Data: In some advanced algorithms (like certain variations of boosting or neural networks), thresholds can be dynamically adjusted based on model performance metrics rather than being statically chosen.
9. Grid Search for Optimal Thresholds
  - Parameter Optimization: In machine learning pipelines, grid search or other optimization techniques can be used to find the best threshold that maximizes a performance metric (like accuracy, F1 score, etc.) on a validation set.

**Hyperparameters for Decision Trees**
- Criterion:
  - Type: String
  - Description: The function to measure the quality of a split.
  - Options:
    - 'gini': Gini impurity (for classification).
    - 'entropy': Information gain (for classification).
    - 'squared_error': Mean squared error (for regression).
    - 'friedman_mse': Friedman’s mean squared error (for regression).
    - 'absolute_error': Mean absolute error (for regression).
    - 'poisson': Poisson deviance (for regression).
- Max Depth:
  - Type: Integer or None
  - Description: The maximum depth of the tree.
  - Default: None (nodes are expanded until all leaves are pure or contain fewer than min_samples_split samples).
- Min Samples Split:
  - Type: Integer or Float
  - Description: The minimum number of samples required to split an internal node.
  - Default: 2.
- Min Samples Leaf:
  - Type: Integer or Float
  - Description: The minimum number of samples that must be present in a leaf node.
  - Default: 1.
- Max Features:
  - Type: Integer, Float, String, or None
  - Description: The number of features to consider when looking for the best split.
  - Options:
    - Integer: Consider max_features features at each split.
    - Float: Consider a fraction of the total number of features.
    - 'auto': Uses sqrt(n_features) for classification and n_features for regression.
    - 'sqrt': Same as 'auto'.
    - 'log2': Consider log2(n_features) features.
    - None: Consider all features.
- Max Leaf Nodes:
  - Type: Integer or None
  - Description: Limits the number of leaf nodes in the tree.
  - Default: None.
- Min Impurity Decrease:
  - Type: Float
  - Description: A node will be split if this value is greater than or equal to the impurity decrease of the node.
  - Default: 0.0.
- Class Weight (for DecisionTreeClassifier):
  - Type: Dict, ‘balanced’, or None
  - Description: Weights associated with classes in classification problems.
  - Default: None (all classes are supposed to have weight one).
- Presort (deprecated in newer versions of scikit-learn):
  - Type: Boolean
  - Description: Whether to presort the data to speed up the finding of the best splits.
  - Default: False.
- Random State:
  - Type: Integer or None
  - Description: Controls the randomness of the estimator.
  - Default: None.

**Stopping Criteria for Decision Trees**
- Maximum Depth: Specify a maximum depth for the tree. This limits how deep the tree can grow, preventing it from becoming overly complex.
- Minimum Samples per Leaf: Set a minimum number of samples (rows) required to be present in a leaf node. If a node has fewer samples than this threshold, it will not be split further.
- Minimum Information Gain: Define a minimum threshold for information gain. If the information gain from a potential split is less than this threshold, the node will not be split.
- Maximum Number of Leaves: Limit the total number of leaf nodes in the tree. This can help control the complexity of the model.
- Pure Node (Homogeneity): Stop splitting when a node is pure, meaning all samples in the node belong to the same class. At this point, no further splits will improve classification.
- Insufficient Gain: If further splits do not provide a significant reduction in impurity (e.g., information gain or Gini impurity), the splitting process can be terminated.
- Predefined Class Distribution: If the distribution of classes in a node reaches a predefined condition (e.g., a specific ratio of classes), further splitting can be stopped.

**Import Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, explained_variance_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

In [2]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, leaf=False, class_value=None):
        self.feature = feature          # Feature index for splitting
        self.threshold = threshold      # Threshold value for splitting
        self.left = left                # Left child node
        self.right = right              # Right child node
        self.leaf = leaf                # Whether this node is a leaf
        self.class_value = class_value  # Class label if it's a leaf

In [3]:
class DecisionTree:
    def __init__(self, criterion='gini', max_depth=None, min_samples_split=2, loss_function='categorical_cross_entropy'):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.loss_function = loss_function
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        # Stopping criteria
        if len(y) < self.min_samples_split or (self.max_depth and depth >= self.max_depth) or len(set(y)) == 1:
            return self._create_leaf_node(y)

        # Find the best split
        best_feature, best_threshold = self._find_best_split(X, y)
        if best_feature is None:
            return self._create_leaf_node(y)

        # Split the data
        left_indices = X[:, best_feature] < best_threshold
        right_indices = X[:, best_feature] >= best_threshold

        left_child = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)

    def _find_best_split(self, X, y):
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(X, y, feature, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _information_gain(self, X, y, feature, threshold):
        # Calculate the impurity before the split
        parent_impurity = self._impurity(y)

        # Split the data
        left_indices = X[:, feature] < threshold
        right_indices = X[:, feature] >= threshold

        # Calculate the weighted impurity of the children
        n = len(y)
        n_left = np.sum(left_indices)
        n_right = np.sum(right_indices)

        if n_left == 0 or n_right == 0:
            return 0

        child_impurity = (n_left / n) * self._impurity(y[left_indices]) + (n_right / n) * self._impurity(y[right_indices])

        # Calculate information gain
        return parent_impurity - child_impurity

    def _impurity(self, y):
        if self.criterion == 'gini':
            return self._gini_impurity(y)
        elif self.criterion == 'entropy':
            return self._entropy(y)

    def _gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        total = len(y)
        return 1 - sum((count / total) ** 2 for count in counts)

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        total = len(y)
        return -sum((count / total) * np.log2(count / total) for count in counts if count > 0)

    def _create_leaf_node(self, y):
        most_common = np.bincount(y).argmax()
        return Node(leaf=True, class_value=most_common)

    def predict(self, X):
        return np.array([self._predict_sample(sample, self.tree) for sample in X])

    def _predict_sample(self, sample, tree):
        if tree.leaf:
            return tree.class_value

        if sample[tree.feature] < tree.threshold:
            return self._predict_sample(sample, tree.left)
        else:
            return self._predict_sample(sample, tree.right)

    def predict_proba(self, X):
        """Predict class probabilities for the input samples."""
        probs = []
        for sample in X:
            class_counts = [0] * (np.max(y) + 1)  # Adjust based on number of classes
            self._count_classes(sample, self.tree, class_counts)
            total = sum(class_counts)
            probs.append([count / total for count in class_counts])  # Convert counts to probabilities
        return np.array(probs)

    def _count_classes(self, sample, tree, class_counts):
        if tree.leaf:
            class_counts[tree.class_value] += 1
            return

        if sample[tree.feature] < tree.threshold:
            self._count_classes(sample, tree.left, class_counts)
        else:
            self._count_classes(sample, tree.right, class_counts)

    def binary_cross_entropy(self, y_true, y_pred):
        epsilon = 1e-9
        y1 = y_true * np.log(y_pred + epsilon)
        y2 = (1 - y_true) * np.log(1 - y_pred + epsilon)
        return -np.mean(y1 + y2)

    def categorical_cross_entropy(self, y_true, y_pred):
        epsilon = 1e-9
        # Assuming y_true is one-hot encoded
        return -np.mean(np.sum(y_true * np.log(y_pred + epsilon), axis=1))

    def calculate_loss(self, y_true, predictions_proba):
        """Calculate the loss based on the selected loss function."""
        if self.loss_function == 'binary_cross_entropy':
            return self.binary_cross_entropy(y_true, predictions_proba[:, 1])  # Assuming binary class 1
        elif self.loss_function == 'categorical_cross_entropy':
            # Convert y_true to one-hot encoding for categorical cross-entropy
            y_true_one_hot = np.eye(np.max(y_true) + 1)[y_true]
            return self.categorical_cross_entropy(y_true_one_hot, predictions_proba)

    def print_tree(self, tree=None, indent="  "):
        """Prints the structure of the decision tree"""
        if tree is None:
            tree = self.tree
        if tree.leaf:
            print(f"{indent}Leaf: Class {tree.class_value}")
        else:
            print(f"{indent}Feature {tree.feature} <= {tree.threshold}")
            print(f"{indent}Left:")
            self.print_tree(tree.left, indent + "  ")
            print(f"{indent}Right:")
            self.print_tree(tree.right, indent + "  ")

    def accuracy(self, y, y_hat):
        return np.sum(y == y_hat) / len(y)

    def custom_metrics(self, y, y_hat):
        tp, tn, fp, fn = 0, 0, 0, 0
        for i in range(len(y)):
            if y[i] == 1 and y_hat[i] == 1:
                tp += 1
            elif y[i] == 1 and y_hat[i] == 0:
                fn += 1
            elif y[i] == 0 and y_hat[i] == 1:
                fp += 1
            elif y[i] == 0 and y_hat[i] == 0:
                tn += 1
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        print(f"Precision: {precision:.2f}")
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        print(f"Recall (Sensitivity): {recall:.2f}")
        f1_score = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
        return f1_score

**Load Dataset**

In [4]:
# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

In [5]:
print(type(X)), print(type(y))
print(X.shape), print(y.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(150, 4)
(150,)


(None, None)

In [6]:
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [8]:
# Train
tree = DecisionTree(criterion='gini', max_depth=3, min_samples_split=2, loss_function='categorical_cross_entropy')
tree.fit(X_train, y_train)
print("Decision Tree Structure:")
tree.print_tree()

Decision Tree Structure:
  Feature 2 <= 3.0
  Left:
    Leaf: Class 0
  Right:
    Feature 2 <= 4.8
    Left:
      Feature 3 <= 1.7
      Left:
        Leaf: Class 1
      Right:
        Leaf: Class 2
    Right:
      Feature 3 <= 1.8
      Left:
        Leaf: Class 1
      Right:
        Leaf: Class 2


In [9]:
# Predictions
predictions = tree.predict(X_test)
print("Predictions:", predictions)
print("Actual:", y_test)

Predictions: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Actual: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]


In [10]:
# Calculate probabilities for loss calculation
predictions_proba = tree.predict_proba(X_test)
print("Prediction Probabilities:\n", predictions_proba)

Prediction Probabilities:
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [11]:
# Loss
loss = tree.calculate_loss(y_test, predictions_proba)
print(f"Loss: {loss:.2f}")

Loss: -0.00


In [12]:
# Accuracy
accuracy = tree.accuracy(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")
f1_score = tree.custom_metrics(y_test, predictions)
print(f"F1 Score: {f1_score:.2f}")

Accuracy: 1.00
Precision: 1.00
Recall (Sensitivity): 1.00
F1 Score: 1.00


In [13]:
# Inbuilt Functions
# Accuracy and Confusion Matrix
accuracy = accuracy_score(y_test, predictions)
print(f"Decision Tree Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, predictions))
print("\nClassification Report:\n", classification_report(y_test, predictions))

Decision Tree Accuracy: 1.00
Confusion Matrix:
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

